# Ejercicios

* Crea una clase abstracta llamada CuentaBancaria que tenga un atributo saldo . Luego, crea dos subclases de CuentaBancaria llamadas CuentaCorriente y CajaAhorro.
 + La clase CuentaCorriente debe tener un atributo adicional llamado limite_descubierto. El constructor debe recibir ese atributo para setearlo
 + la clase CuentaBancaria debe tener un atributo protegido _saldo y una propiedad **saldo** que solo permite acceder en forma de lectura
 + Ambas subclases deben tener métodos **depositar** y **retirar** para manejar el saldo de la cuenta. Depositar tiene que estar definida en CuentaBancaria, y heredada en la subclase, pero Retirar debe ser un metodo abstracto en CuentaBancaria y sobreescrito en las clases hijas.
 + El depósito funciona de la misma manera en ambos casos, pero el retiro en caso de Caja de ahorro no debe permitir llevar el saldo a negativo, mientras que el caso de Cuenta Corriente no debe permitir tener saldo negativo mayor al limite_descubierto.
  + si el retiro en una CuentaCorriente no puede realizarse por el limite de descubierto, debe levantar una excepcion de tipo ValueError
 + El límite de descubierto puede ser modificado (por atributo o propiedad, a elcción).
 + Se debe sobrecargar los operadores + y - que tomaran una cuenta y un monto y funcionarán como deposito y retiro respectivamente
 + La clase CuentaBancaria de tener un metodo de clase **suma_saldo** que devuelva al suma de todos los saldos de todas las cuentas creadas, según con que clase sea invocada, si es invocada en CuentaBancaria, sera la suma de todos los saldos, si es envicocada en CajaAhorro, solamente la suma de las cajas de ahorro, y en CuentaCorriente, solamente la suma de los saldos de las cuenta corrientes, el codigo de esto debe estar completamente resuelto en la clase CuentaBancaria utilizando un atributo privado **__cuentas** para almacenar las cuentas creadas, y de ser posible con una comprehension list para hacer la suma de los saldos


In [ ]:
from abc import ABC, abstractmethod

class CuentaBancaria(ABC):
    __cuentas = []

    @classmethod
    def suma_saldo(cls):
        return sum([i.saldo for i in cls.__cuentas if isinstance(i, cls)])

    def __init__(self):
        self._saldo = 0
        CuentaBancaria.__cuentas.append(self)

    @property
    def saldo(self):
        return self._saldo

    @abstractmethod
    def retirar(self, monto):
        raise NotImplementedError()

    def depositar(self, monto):
        self._saldo += monto

    def __add__(self, second):
        self.depositar(second)

    def __sub__(self, second):
        self.retirar(second)


class CuentaCorriente(CuentaBancaria):
    def __init__(self, limite_descubierto):
      super().__init__()
      self.limite_descubierto = limite_descubierto

    def retirar(self, monto):
      if abs(self._saldo - monto) > self.limite_descubierto:
          raise ValueError()
      self._saldo -= monto

class CajaAhorro(CuentaBancaria):
    def retirar(self, monto):
      if self._saldo < monto:
          raise ValueError()
      self._saldo -= monto


In [ ]:
#@title **Tests para Cuentas Bancarias**

import unittest

class TestCuentaBancaria(unittest.TestCase):
    def tearDown(self):
      CuentaBancaria._CuentaBancaria__cuentas = []

    def test_CB_abstract(self):
      with self.assertRaises(TypeError):
          cb = CuentaBancaria()

    def test_CB_suma_saldo(self):
      ca = CajaAhorro()
      ca._saldo = 100
      ca2 = CajaAhorro()
      ca2._saldo = 300
      cb = CuentaCorriente(limite_descubierto=50)
      cb._saldo = 1000
      cb2 = CuentaCorriente(limite_descubierto=50)
      cb2._saldo = 2000
      self.assertEqual(4, len(CuentaBancaria._CuentaBancaria__cuentas))
      self.assertEqual(3400, CuentaBancaria.suma_saldo())

class TestCuentaCorriente(unittest.TestCase):
    def tearDown(self):
      CuentaBancaria._CuentaBancaria__cuentas = []

    def test_CC_set_saldo_fail(self):
      cc = CuentaCorriente(limite_descubierto=5)
      with self.assertRaises(AttributeError):
          cc.saldo = 330

    def test_CC_constructor_limite_descubierto(self):
      cc = CuentaCorriente(limite_descubierto=5)
      self.assertEqual(5, cc.limite_descubierto)

    def test_CC_modificar_limite_descubierto(self):
      cc = CuentaCorriente(limite_descubierto=5)
      cc.limite_descubierto = 50
      self.assertEqual(50, cc.limite_descubierto)

    def test_CC_retirar_saldo_positivo(self):
      cc = CuentaCorriente(limite_descubierto=50)
      cc._saldo = 150
      cc.retirar(100)
      self.assertEqual(50, cc.saldo)

    def test_CC_retirar_error_limite_descubierto(self):
      cc = CuentaCorriente(limite_descubierto=50)
      with self.assertRaises(ValueError):
          cc.retirar(52)

    def test_CC_retirar_limite_descubierto_justo(self):
      cc = CuentaCorriente(limite_descubierto=50)
      cc.retirar(50)
      self.assertEqual(-50, cc.saldo)

    def test_CC_depositar(self):
      cc = CuentaCorriente(limite_descubierto=50)
      cc.depositar(100)
      self.assertEqual(100, cc.saldo)

    def test_CC_operador_add(self):
      cc = CuentaCorriente(limite_descubierto=50)
      cc + 100
      self.assertEqual(100, cc.saldo)

    def test_CC_operador_sub(self):
      cc = CuentaCorriente(limite_descubierto=50)
      cc - 20
      self.assertEqual(-20, cc.saldo)

    def test_CC_varias_op(self):
      cc = CuentaCorriente(limite_descubierto=50)
      cc.depositar(50)
      cc.retirar(30)
      cc.depositar(10)
      cc.retirar(40)
      cc.retirar(40)
      self.assertEqual(-50, cc.saldo)

    def test_CC_suma_saldo(self):
      ca = CajaAhorro()
      ca._saldo = 100
      ca2 = CajaAhorro()
      ca2._saldo = 300
      cb = CuentaCorriente(limite_descubierto=50)
      cb._saldo = 1000
      cb2 = CuentaCorriente(limite_descubierto=50)
      cb2._saldo = 2000
      self.assertEqual(4, len(CuentaCorriente._CuentaBancaria__cuentas))
      self.assertEqual(3000, CuentaCorriente.suma_saldo())

class TestCajaAhorro(unittest.TestCase):
    def tearDown(self):
      CuentaBancaria._CuentaBancaria__cuentas = []

    def test_CA_set_saldo_fail(self):
      ca = CajaAhorro()
      with self.assertRaises(AttributeError):
          ca.saldo = 330

    def test_CA_retirar_saldo_positivo(self):
      ca = CajaAhorro()
      ca._saldo = 150
      ca.retirar(100)
      self.assertEqual(50, ca.saldo)

    def test_CA_retirar_error_negativo(self):
      ca = CajaAhorro()
      ca._saldo = 20
      with self.assertRaises(ValueError):
          ca.retirar(21)

    def test_CA_retirar_justo(self):
      ca = CajaAhorro()
      ca._saldo = 20
      ca.retirar(20)
      self.assertEqual(0, ca.saldo)

    def test_CA_depositar(self):
      ca = CajaAhorro()
      ca.depositar(100)
      self.assertEqual(100, ca.saldo)

    def test_CA_operador_add(self):
      ca = CajaAhorro()
      ca + 100
      self.assertEqual(100, ca.saldo)

    def test_CA_operador_sub(self):
      ca = CajaAhorro()
      ca._saldo = 50
      ca - 20
      self.assertEqual(30, ca.saldo)

    def test_CA_varias_op(self):
      ca = CajaAhorro()
      ca.depositar(250)
      ca.retirar(30)
      ca.depositar(10)
      ca.retirar(40)
      ca.retirar(40)
      self.assertEqual(150, ca.saldo)

    def test_CA_suma_saldo(self):
      ca = CajaAhorro()
      ca._saldo = 100
      ca2 = CajaAhorro()
      ca2._saldo = 300
      cb = CuentaCorriente(limite_descubierto=50)
      cb._saldo = 1000
      cb2 = CuentaCorriente(limite_descubierto=50)
      cb2._saldo = 2000
      self.assertEqual(4, len(CajaAhorro._CuentaBancaria__cuentas))
      self.assertEqual(400, CajaAhorro.suma_saldo())


unittest.main(argv=[''], verbosity=2, exit=False)


 + Revisar y tratar de entender estas dos formas de definir una clase a la que al construir le puedo pasar parametros con nombre para obtener un objeto con esos atributos accesibles a través del uso del . como si fueran atributos definidos en la instancia.


In [ ]:
class Bunch(object):
  def __init__(self, **kwds):
    self.__dict__.update(kwds)

  def __eq__(self, other):
    return self.__dict__ == other.__dict__


class Bunch2(dict):
  __getattr__, __setatr__ = dict.get, dict.__setitem__

x = Bunch(pepe=23)
print(x.pepe)

print(x)

x = Bunch2(pepe=23,update=33)
print(x.pepe)
print(x)
print(x.update)



Bunch hereda de object, pero simplemente en el constructor, pasa todos los parámetro nominales, y en el constructor lo que se hace es pisar el dict interno que guarda los atributos con los nuevos valores

Bunch2 encambio hereda de dict, porlo que el constructor de dict directamente recibe los parametros nominales, lo que hace es sobreescribir el getter y setter generico para que accedan al get y al set de dict. Un problema con esta idea es que si por casualidad queremos tener un atributo que se llama **update** por ejemplo, no va a andar porque dict tiene un metodo que se llama update.